In [1]:
import base64
import datetime
import json
import os
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim

In [2]:
# Read the data file and take a look at the data
df = pd.read_csv('cleaneddata.txt', sep=",")
df.drop([0], axis=0, inplace=True)
df.drop(['Filler'], axis=1 , inplace = True) #it is all blank information
df.drop(['School Code'], axis=1, inplace = True) #School code is 0000000
df.drop(['Area 1 Percentage Above Standard',
       'Area 1 Percentage Near Standard', 'Area 1 Percentage Below Standard',
       'Area 2 Percentage Above Standard', 'Area 2 Percentage Near Standard',
       'Area 2 Percentage Below Standard', 'Area 3 Percentage Above Standard',
       'Area 3 Percentage Near Standard', 'Area 3 Percentage Below Standard',
       'Area 4 Percentage Above Standard', 'Area 4 Percentage Near Standard',
       'Area 4 Percentage Below Standard'], axis=1, inplace=True)
df.drop(['Test Year','Test Type'], axis=1, inplace=True) #drop because we know is only 2017, test type is B for all
df=df.reset_index()
df.head()
df.drop(['index'], axis = 1, inplace=True)
df.head()
df=df.apply(pd.to_numeric, errors='coerce')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1,2,3,4,5,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
df.head(20) #We can see that we have several grades, several test Id and also several subgroups. We will require to group by some of this columns in order to gather the data for the distsrict.
#To aggregate the outcomes percentages we will convert them to qty first to avoid percentages over 100%

,County Code,District Code,Subgroup ID,Total Tested At Entity Level,Total Tested with Scores,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores,Percentage Tested
0,1,10017,1,51.0,51.0,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10017,1,44.0,44.0,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10017,1,51.0,51.0,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10017,1,44.0,44.0,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10017,1,44.0,44.0,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,10017,1,51.0,51.0,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,10017,1,51.0,51.0,11,2,151.0,44.0,2412.8,0.0,0.00,0.00,4.55,95.45,44.0,100.0
7,1,10017,1,44.0,44.0,11,1,150.0,38.0,2435.4,0.0,5.26,5.26,21.05,73.68,38.0,100.0
8,1,10017,1,51.0,51.0,13,2,177.0,51.0,NaN,0.0,0.00,0.00,3.92,96.08,51.0,100.0
9,1,10017,1,44.0,44.0,13,1,175.0,44.0,NaN,0.0,4.55,4.55,18.18,77.27,44.0,100.0


In [29]:
#To review assumptions, we will confirm the 5 column percentages are representation of 100% of the students.
#After doing this review, it is clear 'Percenage met and above has duplicate information'
#We will only consider the 4 percentage amounts now
df['Percentage Tested']=(df['Percentage Standard Exceeded'] + df['Percentage Standard Met']  + df['Percentage Standard Nearly Met'] + df['Percentage Standard Not Met']).round(decimals=0)

In [45]:
dfp=df['Percentage Tested']
dfp=dfp.dropna()
dfp

6         100.0
7         100.0
8         100.0
9         100.0
16        100.0
17        100.0
18        100.0
19        100.0
24        100.0
25        100.0
26        100.0
27        100.0
34        100.0
35        100.0
36        100.0
37        100.0
54        100.0
55        100.0
56        100.0
57        100.0
64        100.0
65        100.0
66        100.0
67        100.0
86        100.0
87        100.0
88        100.0
89        100.0
112       100.0
113       100.0
          ...  
474910    100.0
474911    100.0
474914    100.0
474915    100.0
474916    100.0
474917    100.0
474938    100.0
474939    100.0
474940    100.0
474941    100.0
474944    100.0
474945    100.0
474946    100.0
474947    100.0
474956    100.0
474957    100.0
474958    100.0
474959    100.0
474960    100.0
474961    100.0
474962    100.0
474963    100.0
474976    100.0
474977    100.0
474978    100.0
474979    100.0
474982    100.0
474983    100.0
474984    100.0
474985    100.0
Name: Percentage Tested,

In [50]:
count=0
for i in dfp:
    if i!=100:
        count=+1
print(count)

0


#After doing this review, it is clear 'Percenage met and above has duplicate information', so we removed it from the calculation of total percentages of students tested.
We will keep the Column for information only.

In [51]:
#Converting all the assessment columns from percentage columns to absolute numbers for exceeded, nearly met, not met
df['Total Standard Exceeded'] =(df['Students Tested']*df['Percentage Standard Exceeded']/100).round(decimals=0)
df['Total Standard Met'] =(df['Students Tested']*df['Percentage Standard Met']/100).round(decimals=0)
df['Total Standard Nearly Met'] =(df['Students Tested']*df['Percentage Standard Nearly Met']/100).round(decimals=0)
df['Total Standard Not Met'] =(df['Students Tested']*df['Percentage Standard Not Met']/100).round(decimals=0)
df['Total Standard Met and Above'] =(df['Students Tested']*df['Percentage Standard Met and Above']/100).round(decimals=0)
#Percentage was multiplied by the students tested information and not the total enrollment information.
#Students tested in Math and Literacy can be the same students or different students, we don't have that information.
#we are adding the results of both exams, this can cause a duplicate in value of students tested, I will separate both datasets

In [52]:
df.head(10)

,County Code,District Code,Subgroup ID,Total Tested At Entity Level,Total Tested with Scores,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,...,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores,Percentage Tested,Total Standard Exceeded,Total Standard Met,Total Standard Nearly Met,Total Standard Not Met,Total Standard Met and Above
0,1,10017,1,51.0,51.0,6,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10017,1,44.0,44.0,6,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10017,1,51.0,51.0,7,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10017,1,44.0,44.0,7,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10017,1,44.0,44.0,8,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,10017,1,51.0,51.0,8,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,10017,1,51.0,51.0,11,2,151.0,44.0,2412.8,...,0.00,4.55,95.45,44.0,100.0,0.0,0.0,2.0,42.0,0.0
7,1,10017,1,44.0,44.0,11,1,150.0,38.0,2435.4,...,5.26,21.05,73.68,38.0,100.0,0.0,2.0,8.0,28.0,2.0
8,1,10017,1,51.0,51.0,13,2,177.0,51.0,NaN,...,0.00,3.92,96.08,51.0,100.0,0.0,0.0,2.0,49.0,0.0
9,1,10017,1,44.0,44.0,13,1,175.0,44.0,NaN,...,4.55,18.18,77.27,44.0,100.0,0.0,2.0,8.0,34.0,2.0


In [53]:
#We will require to create two assessment datasets, one with the total number of students (subgroup id 1) and another with the ethnicity (Subgroup 74.75.76.77.78.79.80) The subgroup 144 
#has two - races. I will bring it over and confirm the addition of subgroup 1 is equal to the ethnicity groups addition.

1   'ALL STUDENTS"
74	"Black or African American"	"Ethnicity"
75	75	"American Indian or Alaska Native"	"Ethnicity"
76	76	"Asian"	"Ethnicity"
77	77	"Filipino"	"Ethnicity"
78	78	"Hispanic or Latino"	"Ethnicity"
79	79	"Native Hawaiian or Pacific Islander"	"Ethnicity"
80	80	"White"	"Ethnicity"
144     "Two or more races"

In [54]:
subgroups_sg= [1,74,75,76,77,78,79,80,144]
#df.loc[df['Language'].apply(lambda x: x in ['C', 'C++'] )]
subgroups_selected = df.loc[df['Subgroup ID'].apply(lambda x: x in subgroups_sg)]
subgroups_selected.head(5)

,County Code,District Code,Subgroup ID,Total Tested At Entity Level,Total Tested with Scores,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,...,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores,Percentage Tested,Total Standard Exceeded,Total Standard Met,Total Standard Nearly Met,Total Standard Not Met,Total Standard Met and Above
0,1,10017,1,51.0,51.0,6,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10017,1,44.0,44.0,6,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10017,1,51.0,51.0,7,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10017,1,44.0,44.0,7,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10017,1,44.0,44.0,8,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
subgroups_selected = subgroups_selected.drop(columns=['Percentage Standard Exceeded','Percentage Standard Met and Above', 'Percentage Standard Met','Percentage Standard Met and Above','Percentage Standard Nearly Met','Percentage Standard Not Met'])

In [56]:
subgroups_selected.columns

Index(['County Code', 'District Code', 'Subgroup ID',
       'Total Tested At Entity Level', 'Total Tested with Scores', 'Grade',
       'Test Id', 'CAASPP Reported Enrollment', 'Students Tested',
       'Mean Scale Score', 'Students with Scores', 'Percentage Tested',
       'Total Standard Exceeded', 'Total Standard Met',
       'Total Standard Nearly Met', 'Total Standard Not Met',
       'Total Standard Met and Above'],
      dtype='object')

In [57]:
df_sub_new=subgroups_selected.groupby(['County Code','District Code','Subgroup ID','Test Id'])['Total Tested At Entity Level', 'Total Tested with Scores', 'CAASPP Reported Enrollment', 'Students Tested',
       'Total Standard Exceeded','Mean Scale Score',
       'Total Standard Met',
       'Total Standard Nearly Met', 'Total Standard Not Met',
       'Students with Scores','Total Standard Met and Above'].sum()

In [58]:
df_sub=df_sub_new.reset_index()

In [60]:
df_sub.head(20)
#This table contains all of the grades together separated by all students and by ethnicities as well as type of test.

,County Code,District Code,Subgroup ID,Test Id,Total Tested At Entity Level,Total Tested with Scores,CAASPP Reported Enrollment,Students Tested,Total Standard Exceeded,Mean Scale Score,Total Standard Met,Total Standard Nearly Met,Total Standard Not Met,Students with Scores,Total Standard Met and Above
0,1,10017,1,1,220.0,220.0,325.0,82.0,0.0,2435.4,4.0,16.0,62.0,82.0,4.0
1,1,10017,1,2,255.0,255.0,328.0,95.0,0.0,2412.8,0.0,4.0,91.0,95.0,0.0
2,1,10017,74,1,105.0,105.0,157.0,41.0,0.0,2419.6,0.0,6.0,35.0,41.0,0.0
3,1,10017,74,2,115.0,115.0,157.0,45.0,0.0,2403.7,0.0,0.0,45.0,45.0,0.0
4,1,10017,76,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,10017,76,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,10017,77,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,10017,77,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,10017,78,1,75.0,75.0,109.0,27.0,0.0,2464.9,2.0,8.0,17.0,27.0,2.0
9,1,10017,78,2,100.0,100.0,110.0,36.0,0.0,2431.3,0.0,2.0,34.0,36.0,0.0


In [64]:
#Now we will create the data set for total students
all_sg= [1]
#df.loc[df['Language'].apply(lambda x: x in ['C', 'C++'] )]
all_students_df = df_sub.loc[df_sub['Subgroup ID'].apply(lambda x: x in all_sg)]
all_students_df=all_students_df.dropna()
all_students_df.head(5)


,County Code,District Code,Subgroup ID,Test Id,Total Tested At Entity Level,Total Tested with Scores,CAASPP Reported Enrollment,Students Tested,Total Standard Exceeded,Mean Scale Score,Total Standard Met,Total Standard Nearly Met,Total Standard Not Met,Students with Scores,Total Standard Met and Above
0,1,10017,1,1,220.0,220.0,325.0,82.0,0.0,2435.4,4.0,16.0,62.0,82.0,4.0
1,1,10017,1,2,255.0,255.0,328.0,95.0,0.0,2412.8,0.0,4.0,91.0,95.0,0.0
16,1,31609,1,1,98.0,98.0,19.0,14.0,1.0,0.0,2.0,8.0,3.0,14.0,3.0
17,1,31609,1,2,91.0,91.0,19.0,13.0,0.0,0.0,0.0,1.0,12.0,13.0,0.0
28,1,31617,1,1,1016.0,1016.0,260.0,244.0,4.0,14301.4,16.0,25.0,199.0,244.0,20.0


In [65]:
#And we will have a separate file separated by ethnicity
subgroups_sg= [74,75,76,77,78,79,80,144]
#df.loc[df['Language'].apply(lambda x: x in ['C', 'C++'] )]
ethnicity_df= df_sub.loc[df_sub['Subgroup ID'].apply(lambda x: x in subgroups_sg)]
ethnicity_df=ethnicity_df.dropna()
ethnicity_df.head(5)

,County Code,District Code,Subgroup ID,Test Id,Total Tested At Entity Level,Total Tested with Scores,CAASPP Reported Enrollment,Students Tested,Total Standard Exceeded,Mean Scale Score,Total Standard Met,Total Standard Nearly Met,Total Standard Not Met,Students with Scores,Total Standard Met and Above
2,1,10017,74,1,105.0,105.0,157.0,41.0,0.0,2419.6,0.0,6.0,35.0,41.0,0.0
3,1,10017,74,2,115.0,115.0,157.0,45.0,0.0,2403.7,0.0,0.0,45.0,45.0,0.0
4,1,10017,76,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,10017,76,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,10017,77,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
all_students_df.to_csv(r'20201220/all_assessment.csv', index = False, header=True)
ethnicity_df.to_csv(r'20201220/ethnicity_new.csv', index = False, header=True)